In [2]:
#Nettoyer un ficher texte pour le lire après

filename ="test.txt"
def txt_propre(filename, new="test.txt"):
    f1 = open(filename, "r")
    lines = f1.readlines()
    f1.close()
    f2 = open(new, "w")
    for l in lines:
        f2.write(l.strip())
    else:
        f2.close()
    return True

In [3]:
# Fonctions utiles

def translate_complement(Sequence):
    # takes a string of ACTG and returns the complementary code
    newSequence = ""
    for letter in Sequence:
        if letter == "A":
            newSequence += "T"
        elif letter == "T":
            newSequence += "A"
        elif letter == "C":
            newSequence += "G"
        elif letter == "G":
            newSequence += "C"
        else:
            print("Error : unrecognized character " ,letter)
    return newSequence

def invert_string(txt):
    return txt[::-1]

def complement(Sequence):
    return invert_string(translate_complement(Sequence))


In [4]:
#Extraire tous les ORF

def openchrfile(filename):
    chromosomef = open(filename,"r")
    chromosome = chromosomef.read()
    chromosomef.close()
    return chromosome

def extract_ORFs(pluschromosome):
    # in : string of DNA
    # out : List of all sequences between start and stop codons for normal and reverse sequences
    minuschromosome = complement(pluschromosome)
    reading = False
    List = []
    for chromosome in [pluschromosome, minuschromosome]:
        print("length :",len(chromosome))
        c= 0
        for i in range(len(chromosome)-4):  
            if chromosome[i] == "A" and chromosome[i+1] == "T" and chromosome[i+2] == "G":
                reading = True
                j = 0
                gene = []
                while reading == True:
                    if chromosome[i+j:i+j+3] == "TAG" or chromosome[i+j:i+j+3] == "TGA" or chromosome[i+j:i+j+3] == "TAA":
                        reading = False
                        gene = chromosome[i:i+j+3]
                        List.append(gene)
                        c+=1
                    elif i+j-3 > len(chromosome):
                        reading = False
                    j += 3
        print( "extract_ORFs: found", c, "ORFs in sequence")
    return List

In [5]:
#Elimine les ORF plus petits qu'une certaine valeur

def threshold_orf(List, threshold):
    # Takes a list of ORFs and outputs a list of ORFs without those shorter than threshold value

    Newlist = []
    for sequence in List:
        if len(sequence) >= threshold:
            Newlist.append(sequence)
    print("threshold_ORFs:",len(Newlist) , "ORFs above", threshold, "bp length, out of", len(List),"initial list")
    
    return Newlist
        
    

In [6]:
def getsequence(start,stop,chro):
    #returns the the part of a string between start and stop values of a longer string
    return chro[start-1:stop]


In [7]:
def extract_genes(filename,chromosome):
# in : .csv file of gene list from geneviewer containing gene indices 
# in2: a string from which gene sequences will be extracted
# out : vector of lists of + and - gene sequences

    fileindexes = open(filename,'r')
    fileIndexes = fileindexes.readlines()
    fileindexes.close()

    #remove the 2 first lines (useless)
    geneList = []
    for line in fileIndexes[2:]:
        new = line.split(",")
        geneList.append(new[:-1])
    #genelist (Name , start, stop , name, +- , ?)
    p=0
    m=0
    seqgeneList = []
    for gene in geneList:
        if gene[4] == 'plus':
            seqgeneList.append(getsequence(int(gene[1]),int(gene[2]),chromosome))
            p += 1
        if gene[4] == 'minus':     
            seqgeneList.append(complement(getsequence(int(gene[1]),int(gene[2]),chromosome)))
            m += 1
    print ("extract_genes:",p, "plus and",m , "minus genes found")
    return seqgeneList

#print(chr1geneList[58])

In [8]:
def seqGeneMatch(ORFList,GeneList):
    # returns a dictionnary in which every sequence is labeled as "gene" if found in the Genelist, else "noise"
    seqDictionnary = {}
    c = 0
    d = 0
    repeats = {}
    for seq in ORFList:
        for gene in GeneList:
            if seq == gene:
                seqDictionnary[seq] = "gene"
                c += 1
                break
            else:
                seqDictionnary[seq] = "noise"
    print("From a list of ", len(ORFList), "ORFs, we found " , c , " genes out of a library of", len(GeneList), "genes, and ","?", "were genes found more than once")
    return seqDictionnary, c




In [9]:
def labelledDatafromfile(CrudeChromosomeFile,CSVindicesFile):
    txt_propre(CrudeChromosomeFile, "chr.txt")
    pluschromosome = openchrfile("chr.txt")
    orfList = extract_ORFs(pluschromosome)
    orfList = threshold_orf(orfList,30)
    geneList = extract_genes(CSVindicesFile,pluschromosome)
    return seqGeneMatch(orfList,geneList)

data,size = labelledDatafromfile("test.txt","genechr1.csv")

length : 230218
extract_ORFs: found 4297 ORFs in sequence
length : 230218
extract_ORFs: found 4265 ORFs in sequence
threshold_ORFs: 5778 ORFs above 30 bp length, out of 8562 initial list
extract_genes: 55 plus and 46 minus genes found
From a list of  5778 ORFs, we found  91  genes out of a library of 101 genes, and  ? were genes found more than once


In [10]:
def createlibrary(data,size):
    # takes dictionnary and outputs two vectors ready for Deeplearning
    sequences = []
    labels = []
    for seq in data.keys():
        if data[seq] == "gene":
            sequences.append(seq)
            labels.append(data[seq])
    n=0
    for seq in data.keys():
        if n>size:
            break
        elif label == "noise":
            n += 1
            sequences.append(seq)
            labels.append(label)
    
    return sequences, labels


In [11]:
def treatdata():
    ListSequences = []
    ListLabels = []
    for i  in range(15):
        a = "propre_chr_" + str(i+2) +".txt"
        b = "Genes_" + str(i+2) + ".csv"
        data, size = labelledDatafromfile(a,b)
        sequences , labels = createlibrary(data,size)
        ListSequences += sequences
        ListLabels += labels
        
    f = open("dataTrain.txt", 'w')
    for i in range(len(ListSequences)):
        f.write(ListSequences[i] + "," + ListLabels[i] + "\n")
    f.close()
        
treatdata()     

length : 813184
extract_ORFs: found 14458 ORFs in sequence
length : 813184
extract_ORFs: found 15352 ORFs in sequence
threshold_ORFs: 19833 ORFs above 30 bp length, out of 29810 initial list
extract_genes: 203 plus and 228 minus genes found
From a list of  19833 ORFs, we found  385  genes out of a library of 431 genes, and  ? were genes found more than once


NameError: name 'label' is not defined